### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [ ]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

'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'

In [23]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EE5B500E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EE5B501810>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [25]:
result

AIMessage(content='Here are a few ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, used with someone you know well)\n* **Coucou, ça va ?** (Very informal, used with close friends) \n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 21, 'total_tokens': 111, 'completion_time': 0.163636364, 'prompt_time': 0.002074629, 'queue_time': 0.24828569100000003, 'total_time': 0.165710993}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c5b02240-b3be-4090-a08e-41fdff35f825-0', usage_metadata={'input_tokens': 21, 'output_tokens': 90, 'total_tokens': 111})

In [26]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, used with someone you know well)\n* **Coucou, ça va ?** (Very informal, used with close friends) \n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [27]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

"Hello - Bonjour\n\nHow are you? - Comment allez-vous ? (formal) or Comment vas-tu ? (informal) \n\n\nLet me know if you'd like to translate anything else! 😊\n"

In [ ]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [34]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [30]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [31]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [ ]:
chain.invoke({"language":"Urdu","text":"How are you?"})

'The most common way to say "How are you?" in Urdu is:\n\n**کیسے ہیں؟** (Kaisay hain?) \n\nThis is a general greeting used for both formal and informal situations. \n\n\nHere are some other variations:\n\n* **आप कैसे हैं?** (Aap kaise hain?) - This is a more formal way to ask "How are you?" and is used when addressing someone older or of higher status.\n* **کیسے چلے ہیں؟** (Kaisay chalay hain?) - This translates to "How are things going?" \n\n\nHope this helps!\n'